In [ ]:
import mplhep as hep
import uproot
from egamma_tnp import TagNProbe
from egamma_tnp.plot import plot_efficiency
from matplotlib import pyplot as plt

hep.style.use("CMS")
hep.style.use({"font.size": 16})
hep.style.use({"figure.figsize": (6.4, 4.8)})

In [ ]:
from distributed import Client

# from lpcjobqueue import LPCCondorCluster

# cluster = LPCCondorCluster(ship_env=True)
# cluster.adapt(minimum=1, maximum=100)
client = Client()

In [ ]:
%%time

# tag_n_probe = TagNProbe(
#     [
#         "/EGamma0/Run2023C-PromptNanoAODv12_v3-v1/NANOAOD",
#         "/EGamma1/Run2023C-PromptNanoAODv12_v3-v1/NANOAOD",
#     ],
#     32,
#     goldenjson="json/Cert_Collisions2023_366442_368823_Golden.json",
#     toquery=True,
#     redirect=False,
# )

tag_n_probe = TagNProbe(
    ["root_files/Egamma0.root", "root_files/Egamma1.root"],
    32,
    goldenjson="json/Cert_Collisions2023_366442_370790_Golden.json",
    toquery=False,
    redirect=False,
)


tag_n_probe.remove_bad_xrootd_files(
    [
        "root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2023C/EGamma0/NANOAOD/PromptNanoAODv12_v4-v1/70000/a1e1c26c-4328-4147-a247-cb8fa85c0d04.root",
        "root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/data/Run2023C/EGamma1/NANOAOD/PromptNanoAODv12_v4-v1/2810000/c0e3ecf9-d65e-440b-acb9-af89a073a217.root",
    ]
)
tag_n_probe.load_events()
tag_n_probe

In [ ]:
%%time

(
    hpt_pass,
    hpt_all,
    heta_pass,
    heta_all,
) = tag_n_probe.get_tnp_histograms(compute=True, scheduler=None, progress=True)

print(f"Passing probes: {hpt_pass.sum(flow=True)}")
print(f"All probes: {hpt_all.sum(flow=True)}")

In [ ]:
with uproot.recreate("root_files/histograms.root") as file:
    file["hpt_pass"] = hpt_pass
    file["hpt_all"] = hpt_all
    file["heta_pass"] = heta_pass
    file["heta_all"] = heta_all

In [ ]:
with uproot.open("root_files/histograms.root") as file:
    hpt_pass = file["hpt_pass"].to_hist()
    hpt_all = file["hpt_all"].to_hist()
    heta_pass = file["heta_pass"].to_hist()
    heta_all = file["heta_all"].to_hist()

print(f"Passing probes: {hpt_pass.sum(flow=True)}")
print(f"All probes: {hpt_all.sum(flow=True)}")

In [ ]:
hpt_pass.plot1d(label="passing probes", flow="none")
hpt_all.plot1d(label="all probes", flow="none")
plt.xlim(5, 400)
plt.xlabel(r"$P_T$ [GeV]")
plt.ylabel(r"Counts")
plt.legend()
plt.savefig("plots/pt.pdf", bbox_inches="tight")
hep.cms.label("Preliminary", data=True, lumi="X", year=2023, com=13.6)
plt.show()

In [ ]:
heta_pass.plot1d(label="passing probes", flow="none")
heta_all.plot1d(label="all probes", flow="none")
plt.xlim(-2.5, 2.5)
plt.xlabel(r"$\eta$")
plt.ylabel(r"Counts")
plt.legend()
plt.savefig("plots/eta.pdf", bbox_inches="tight")
hep.cms.label("Preliminary", data=True, lumi="X", year=2023, com=13.6)
plt.show()

In [ ]:
plot_efficiency(hpt_pass, hpt_all)
hep.cms.label("Preliminary", data=True, lumi="X", year=2023, com=13.6)
plt.xlim(5, 400)
plt.xlabel(r"$P_T$ [GeV]")
plt.ylabel(r"Efficiency")
plt.xscale("log")
plt.savefig("plots/eff_pt.pdf")
plt.show()

In [ ]:
plot_efficiency(heta_pass, heta_all)
hep.cms.label("Preliminary", data=True, lumi="X", year=2023, com=13.6)
plt.xlim(-2.5, 2.5)
plt.xlabel(r"$\eta$")
plt.ylabel(r"Efficiency")
plt.savefig("plots/eff_eta.pdf")
plt.show()